In [144]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import paths
import shutil
from PIL import Image
import math
from os.path import join
from os import listdir
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline

In [160]:
k_ids = ['kfold_0_ids.npy', 'kfold_1_ids.npy', 'kfold_2_ids.npy', 
       'kfold_3_ids.npy', 'kfold_4_ids.npy']
k_preds = ['kfold_0_preds.npy', 'kfold_1_preds.npy', 'kfold_2_preds.npy', 
       'kfold_3_preds.npy', 'kfold_4_preds.npy']
test_ids_path = 'cnn_test_filenames.npy'
test_preds_path = 'cnn_test_preds.npy'

In [147]:
train_json = pd.read_json(paths.TRAIN_JSON)
listing_ids = train_json.loc[:,'listing_id']
test_json = pd.read_json(paths.TEST_JSON)
test_listing_ids = test_json.loc[:,'listing_id']

In [110]:
# concat all of the k fold predictions
index =0
X_trn = pd.DataFrame()
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for trn_indices, val_indices in kf.split(range(len(listing_id_interest))):
    val_ids = listing_ids.iloc[val_indices]
    curr_ids = np.load(join(paths.CNN_OUTPUT_DIR, k_ids[index]))
    curr_preds = np.load(join(paths.CNN_OUTPUT_DIR, k_preds[index]))
    curr_preds = curr_preds[:len(curr_ids)]
    temp = pd.DataFrame([curr_ids, curr_preds[:,0], curr_preds[:,1], 
                         curr_preds[:,2]])
    temp.rename(index = {0:'listing_id', 
                           1:'low', 
                           2:'medium', 
                           3:'high'}, inplace=True)  
    temp = temp.T
    temp['listing_id'] = temp['listing_id'].apply(lambda x: 
                                x[x.index('/')+1:x.index('_')])
    X_trn = pd.concat([X_trn, temp])
    index += 1  

In [161]:
test_ids = np.load(join(paths.CNN_OUTPUT_DIR, test_ids_path))
test_preds = np.load(join(paths.CNN_OUTPUT_DIR, test_preds_path))
test_preds = test_preds[:len(test_ids)]
X_test = pd.DataFrame([test_ids, test_preds[:,0], test_preds[:,1], 
                         test_preds[:,2]])
X_test.rename(index = {0:'listing_id', 
                           1:'low', 
                           2:'medium', 
                           3:'high'}, inplace=True)  
X_test = X_test.T
X_test['listing_id'] = X_test['listing_id'].apply(lambda x: 
                                x[x.index('/')+1:x.index('_')])

[[ 0.59359705  0.24123178  0.16517116]
 [ 0.71115565  0.28340146  0.00544293]
 [ 0.60366648  0.27263457  0.12369898]
 [ 0.97617412  0.02194566  0.00188023]
 [ 0.56468087  0.22830351  0.20701556]
 [ 0.67674303  0.19292016  0.13033681]
 [ 0.62202752  0.1668323   0.21114013]
 [ 0.60231316  0.25946197  0.13822484]
 [ 0.59520853  0.3275767   0.07721476]
 [ 0.67520541  0.25742984  0.06736482]]


In [163]:
# group by
# X_trn = X_trn.apply(pd.to_numeric)
# X_grouped = X_trn.groupby('listing_id').mean()
X_test = X_test.apply(pd.to_numeric)
X_test_grouped = X_test.groupby('listing_id').mean()

In [168]:
# X_merged = pd.DataFrame(listing_ids)
# X_merged = X_merged.join(X_grouped, on='listing_id', how='left')
X_test_merged = pd.DataFrame(test_listing_ids)
X_test_merged = X_test_merged.join(X_test_grouped, 
                                   on='listing_id', how='left')

In [166]:
X_merged.head()

,listing_id,low,medium,high
10,7211212,0.572419,0.378158,0.049423
10000,7150865,0.724667,0.190186,0.085146
100004,6887163,0.687312,0.236295,0.076393
100007,6888711,0.751953,0.209664,0.038383
100013,6934781,0.832966,0.133754,0.033280


In [143]:
X_merged.to_csv(join(paths.CNN_OUTPUT_DIR,"cnn_train.csv"))

In [169]:
X_test_merged.head()

,listing_id,low,medium,high
0,7142618,0.571914,0.323286,0.104800
1,7210040,0.781394,0.133138,0.085467
100,7103890,0.677084,0.239322,0.083595
1000,7143442,0.764812,0.180614,0.054574
100000,6860601,0.610674,0.334905,0.054421


In [170]:
X_test_merged.to_csv(join(paths.CNN_OUTPUT_DIR,"cnn_test.csv"))